### Importing libraries

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

### Reading Data

In [2]:
df = pd.read_csv("new_Base_CDM_balanced_V2.csv")